In [1]:
import pandas as pd
import os
import re
import numpy as np
from statistics import stdev
import math

In [2]:
def find_cord(image_name,idx):
    return re.findall(r'\d+\.\d+',image_name)[idx]

def getVerDisStd(df,rm_outlier=True):
    ''' Get standard deviation of consecutive vertical difference '''
    df = df[df['prob']>=0.45]
    df.reset_index(drop=True,inplace=True)
    df['pred']=df['pred'].str.strip()
    df['h_level'] = (df['new_3y']+df['new_4y'])/2
    
    # Series of difference between the consecutive values        
    gaps = [y - x for x, y in zip(df['h_level'][:-1], df['h_level'][1:])]
    std = stdev(gaps)
    if rm_outlier:
        pstl_90 = np.quantile(gaps,0.9)
        gaps_90pstl = [i for i in gaps if i <90]
        std = stdev(gaps_90pstl)

    return std,df

def write_menu(df,file_name):
    
    std,df = getVerDisStd(df)
    
    ''' Write menu '''
    list_line = [[df.loc[0,'new_4x']]]
    counter=1
    for x in df['h_level'][1:]:
        # if the gap from the current item to the previous is more than 1 SD
        # Note: the previous item is the last item in the last list
        # Note: modify '> 0.6' to adjust threshold for separating lines
        if (x-df.loc[counter-1,'h_level']) / std > 0.6:
            list_line.append([])
        list_line[-1].append(df.loc[counter,'new_4x'])
        counter+=1
    
    menu_name=file_name.split('.')[0][:-12]+'_new.txt'
    menu = open('../data/output/'+menu_name,"w") 
    # Decode sentence and write it to menu file
    word_count=0
    for line in list_line:
        # sort words with x coordinate of vertex 4 
        line.sort()
        line_in_preds=[]
        len_sentence = len(line)

        # decode menu content from x coordinate of vertex 4 
        for word in line:
            df_sub_conf = df.loc[word_count:word_count+len_sentence]
            df_sub_conf[df_sub_conf['new_4x']==word]['pred'].values[0]
            line_in_preds.append(df_sub_conf[df_sub_conf['new_4x']==word]['pred'].values[0])

        # update counter using length of sentence
        word_count+=len_sentence

        # ordered list in prediction word format to menu file
        sentence = ' '.join(line_in_preds)
        menu.write(sentence)
        menu.write('\n')
    menu.close()

In [3]:
pred_list = [os.path.join('../data/output/', f) for f in os.listdir('../data/output/') if os.path.isfile(os.path.join('../data/output/', f)) and f.endswith('pred_result.txt')]
for pred_file in pred_list:
    raw_log = pd.read_csv(pred_file,sep="\t", header = None)
    result=raw_log.copy()
    result.columns=['image_name','pred','prob']
    result['vertex1_x']=result.apply(lambda x: find_cord(x.image_name, 0), axis=1).astype(float)
    result['vertex1_y']=result.apply(lambda x: find_cord(x.image_name, 1), axis=1).astype(float)
    result['vertex2_x']=result.apply(lambda x: find_cord(x.image_name, 2), axis=1).astype(float)
    result['vertex2_y']=result.apply(lambda x: find_cord(x.image_name, 3), axis=1).astype(float)
    result['vertex3_x']=result.apply(lambda x: find_cord(x.image_name, 4), axis=1).astype(float)
    result['vertex3_y']=result.apply(lambda x: find_cord(x.image_name, 5), axis=1).astype(float)
    result['vertex4_x']=result.apply(lambda x: find_cord(x.image_name, 6), axis=1).astype(float)
    result['vertex4_y']=result.apply(lambda x: find_cord(x.image_name, 7), axis=1).astype(float)
    result.reset_index(inplace=True)
    result['rotation_x']=result['vertex4_x'][0]
    result['rotation_y']=result['vertex4_y'][0]
    result['ratio']=(result['vertex4_y']-result['vertex3_y'])/(result['vertex4_x']-result['vertex3_x'])
    result['theta']=result['ratio'].apply(lambda x: math.atan(x))
    result['theta'].replace(0,np.NaN,inplace=True)
    result['correction_theta']=result['theta'].apply(lambda x:-x).mean()
    result['cos_theta']=result['correction_theta'].apply(lambda x: math.cos(x))
    result['sin_theta']=result['correction_theta'].apply(lambda x: math.sin(x))
    result['new_3x']=(result['vertex3_x']-result['rotation_x'])*result['cos_theta']-(result['vertex3_y']-result['rotation_y'])*result['sin_theta']+ result['rotation_x']
    result['new_3y']=(result['vertex3_x']-result['rotation_x'])*result['sin_theta']+(result['vertex3_y']-result['rotation_y'])*result['cos_theta']+ result['rotation_y']
    result['new_4x']=(result['vertex4_x']-result['rotation_x'])*result['cos_theta']-(result['vertex4_y']-result['rotation_y'])*result['sin_theta']+ result['rotation_x']
    result['new_4y']=(result['vertex4_x']-result['rotation_x'])*result['sin_theta']+(result['vertex4_y']-result['rotation_y'])*result['cos_theta']+ result['rotation_y']

    result=result[['image_name','vertex1_x','vertex1_y','vertex2_x','vertex2_y','vertex3_x','vertex3_y','vertex4_x','vertex4_y','pred','prob','rotation_x','rotation_y','ratio','theta','correction_theta','cos_theta','sin_theta','new_3x','new_3y','new_4x','new_4y']]
    result['pred']=result['pred'].str.strip()
    df=result[(result['prob']<0.3) & (result['pred'].str.contains('\$?\d+', regex=True))]['pred'].apply(lambda x: x[:-1])
    result.update(df)
    result['pred']=result['pred'].apply(lambda x:x.split('-')[0])
    print(result.loc[(result['prob']<0.3) & (result['pred'].str.contains('\$?\d+', regex=True)),['image_name','pred','prob']])
    result.sort_values('new_4y',inplace=True)
    
    # prepare input for write_menu()
    file_name=pred_file.split('/')[-1]
    df=result.copy()
#     write_menu(df,file_name)
    

                                           image_name pred    prob
40  image_1_110.05312_763.3172_131.2172_763.3172_1...   45  0.2561
57  image_1_129.80624_316.05_160.84688_316.05_160....  180  0.1603
62  image_1_139.68282_447.26718_169.3125_447.26718...  180  0.2866
70  image_1_156.61406_816.9328_179.18906_816.9328_...   50  0.0542
76  image_1_177.77812_610.9359_207.4078_610.9359_2...  180  0.2472
78  image_1_179.18906_101.5875_213.05156_101.5875_...  250  0.2703
79  image_1_182.01094_529.10156_211.64062_529.1015...  190  0.2444
83  image_1_204.58594_675.83905_238.44844_675.8390...  240  0.2688
                                           image_name  pred    prob
57  image_2_124.1625_684.3047_155.20312_684.3047_1...  1000  0.1796
64  image_2_136.86093_396.47345_166.49063_396.4734...   150  0.2367
70  image_2_153.79219_331.5703_184.83281_331.5703_...  1300  0.1847
82  image_2_183.42188_595.41565_207.4078_595.41565...    90  0.2694
84  image_2_187.6547_767.55_220.10625_767.55_220.1...  10

In [7]:
std,df = getVerDisStd(df)
    
''' Write menu '''
list_line = [[df.loc[0,'new_4x']]]
counter=1
for x in df['h_level'][1:]:
        # if the gap from the current item to the previous is more than 1 SD
        # Note: the previous item is the last item in the last list
        # Note: modify '> 0.6' to adjust threshold for separating lines
    if (x-df.loc[counter-1,'h_level']) / std > 0.6:
        list_line.append([])
    list_line[-1].append(df.loc[counter,'new_4x'])
    counter+=1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
